# トピックモデル

テキストの分析、あるいはテキストマイニングの代表的な分析手法に **トピックモデル** があります。トピックは、テーマあるいは主題とも表現されます。
新聞を例に説明しましょう。もっとも、今どきは新聞を購読していない人も多いと思いますが、新聞を読んだことが一度もないという人まだ少数派でしょう。さて、新聞の記事には、経済、政治、科学、娯楽、スポーツなどの種類があります。これがトピックに相当します。また、あるトピックには出現しやすい単語というのがあります。
政治に関する記事であれば「国会」や「選挙」といった単語が多く出現することになりますし、エンターテインメントであれば「タレント」とか「ドラマ」といった単語が目に付くでしょう。ただし、「タレント」という単語が政治ジャンルには出現しないということはありません。タレントから政治家に転身する人は多いのですから、そうした記事では政治とエンターテインメントそれぞれに顕著な単語の両方が出現していると考えられます。


## LDA

**LDA** は確率モデルです。確率とは、単語が出現する確率になります。テキストごとに単語が出現する確率を推定します。すると、一部のテキスト集合でだけ高い確率で出現する単語の集合が明らかになることがあります。高確率で出現する単語の集合がトピックを構成するとみなすわけです。この単語の集合に政治関連の言葉が多数含まれていれば、その単語集合は「政治トピック」を構成すると解釈できることになります。

In [1]:
## https://github.com/yuukimiyo/GeneralPolicySpeechOfPrimeMinisterOfJapan
## からダウンロードしたファイルを解凍し、longfilenameフォルダにある
## utf8フォルダを data 直下に配置
import os
import re
files = ['data/utf8/' + path for path in os.listdir('data/utf8')]
## ファイル名から国会の番号と首相の名前を抽出
pattern = 'data/utf8/\\d{8}_(\\d{1,3}_[a-z]{1,}-[a-z]{1,})_general-policy-speech.txt'
results = [re.match(pattern, file_name) for file_name in files]
prime_names = [ res.group(1) for res in results]

In [2]:
stopwords = [],
with open('data/stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = [w.strip() for w in f]
## ストップワードをさらに追加
stopwords.extend(["あの", "この", "ある", "する", "いる", "できる", "なる", "れる", "の", "は", "〇", "ソ", "もつ", "わが国", "われわれ","私たち","そのため","行なう","おこなう"])
## セットに変更（形態素が重複して登録されているのを避けるため）
stopwords = set(stopwords)
## ストップワードの要素数を確認
print(len(stopwords))

330


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
## 形態素解析器としてMeCabを指定
import my_mecab_stopwords as my_tokenizer
args={'stopwords_list': stopwords}
## フォルダからファイルを読み込んで辞書を作成
vectorizer = CountVectorizer(input='filename', lowercase=False,
                             max_df=0.5, max_features=200,
                             tokenizer=lambda text: my_tokenizer.tokens(text, **args))
## 文書単語行列を生成
prime_dtm = vectorizer.fit_transform(files)


LDA を適用すべき単語文書行列が生成されました。

In [4]:
from sklearn.decomposition import LatentDirichletAllocation
lda = LatentDirichletAllocation(
    ## トピック数
    n_components=5,
    ## 推定における計算回数
    max_iter=20,
    ## 乱数の種を指定
    random_state = 123
)
topic_data = lda.fit_transform(prime_dtm)

トピックが推定されました。まず、それぞれのトピックで頻出する語を確認してみましょう。

In [5]:
## トピックごとに高頻度で現れる単語の一覧
features = vectorizer.get_feature_names()
## トピック数として5を指定した
for tn in range(5):
    print("topic number: " + str(tn))
    ## トピックごとに上位20語を表示
    row = lda.components_[tn]
    words = ', '.join([features[i] for i in row.argsort()[:-20-1:-1]])
    print(words, "\n")

topic number: 0
議論, 沖縄, 情報, 皆様, 安心, 構築, システム, 医療, 年金, 平成, 戦略, 再生, ふさわしい, 科学, 子供, 連携, 介護, 取り組み, サービス, 徹底 

topic number: 1
物価, はかる, 所存, 存じる, 沖, 石油, 繩, 収支, エネルギー, 上昇, 選挙, 諸君, 需要, 均衡, 資源, 輸入, 輸出, 事態, 価格, 次第 

topic number: 2
被災, 再生, 皆さん, 復興, 戦略, 安心, 危機, 皆様, エネルギー, 取り戻す, 声, 支える, 医療, 事故, 日本人, 被害, つくる, 全国, 連携, 政権 

topic number: 3
国連, 役割, 資金, 太平洋, 構築, 関連, 事件, 地球, 深刻, 政権, 活性, 緊急, 機関, 皆さん, 情報, 秩序, 取り組み, 事態, 市場, 核 

topic number: 4
皆様, 税, 選挙, 所存, サミット, 是正, 次第, 最大限, 向かう, 行財政, 調和, 国土, 展開, 人類, 臨時, 昭和, 至る, 方向, 配慮, 東京 



/mnt/2bddf92b-47f9-4809-95a5-b91e7f25af27/myData/GitHub/myPython/lib/python3.10/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


各トピックにおいて出現確率の高い単語を上位から20語だけ抽出してみました。また、トピックモデルでは、それぞれの所信表明演説において、5つのトピックが占める割合も推定されています。確認のため10の演説に絞って、トピックの割合を確認してみます。


In [6]:
for i,lda in enumerate(topic_data[:10]):
    topicid=[j for j, x in enumerate(lda) if x == max(lda)]
    print('speech = ' + prime_names [i]  + ' : estimated = ' + str(lda)  + ' : max topic = ' + str(topicid[0]))

speech = 47_sato-eisaku : estimated = [0.00128637 0.99482576 0.00129781 0.00128914 0.00130092] : max topic = 1
speech = 185_abe-shinzo : estimated = [0.13387758 0.00146361 0.7179668  0.00146544 0.14522657] : max topic = 2
speech = 26_kishi-nobusuke : estimated = [0.01282082 0.5789924  0.01262974 0.01291338 0.38264367] : max topic = 1
speech = 163_koizumi-jyunichiro : estimated = [0.35231481 0.09198511 0.29139269 0.26179266 0.00251473] : max topic = 0
speech = 49_sato-eisaku : estimated = [0.00168687 0.99325707 0.00168015 0.00167989 0.00169602] : max topic = 1
speech = 25_hatoyama-ichiro : estimated = [0.00552572 0.97788474 0.00548818 0.00558409 0.00551727] : max topic = 1
speech = 70_tanaka-kakuei : estimated = [0.0306515  0.94730687 0.00152412 0.01899482 0.00152269] : max topic = 1
speech = 53_sato-eisaku : estimated = [0.00161257 0.93651218 0.00161758 0.05863828 0.00161939] : max topic = 1
speech = 90_ohira-masayoshi : estimated = [0.00177081 0.36724393 0.0017624  0.00177    0.627452

In [7]:
import pandas as pd
df = pd.DataFrame(columns=['speech', 'topic', 'ratio'])
for i,lda in enumerate(topic_data):
    topicid=[j for j, x in enumerate(lda) if x == max(lda)]
    df = df.append({'speech': prime_names [i], 'topic': topicid[0], 'ratio': max(lda)}, ignore_index=True)

/tmp/ipykernel_61913/702585598.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'speech': prime_names [i], 'topic': topicid[0], 'ratio': max(lda)}, ignore_index=True)
/tmp/ipykernel_61913/702585598.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'speech': prime_names [i], 'topic': topicid[0], 'ratio': max(lda)}, ignore_index=True)
/tmp/ipykernel_61913/702585598.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'speech': prime_names [i], 'topic': topicid[0], 'ratio': max(lda)}, ignore_index=True)
/tmp/ipykernel_61913/702585598.py:5: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df 

作成されたデータフレームの冒頭を確認しましょう。

In [8]:
df.head()

,speech,topic,ratio
0,47_sato-eisaku,1,0.994826
1,185_abe-shinzo,2,0.717967
2,26_kishi-nobusuke,1,0.578992
3,163_koizumi-jyunichiro,0,0.352315
4,49_sato-eisaku,1,0.993257


佐藤栄作氏の第47回国会での演説ではトピック3番（添字で`[2]`となります）が約87パーセントを占めることがわかります。一方、安倍晋三氏の第185回国会での演説では約59パーセントが1番目のトピックで占められています。
以下、トピックの分類結果から、5種類のトピックのいずれが高い割合で出現しているかを調べ、そのパーセンテージが高い順に5個程度抜き出してみましょう。

In [10]:
for i in range(5):
    print(df.query('topic == @i').sort_values(['ratio', 'speech'], ascending=False).head(5))

                    speech topic     ratio
22  153_koizumi-jyunichiro     0  0.927428
81        150_mori-yoshiro     0  0.909243
80        149_mori-yoshiro     0  0.904446
17  157_koizumi-jyunichiro     0  0.884051
58  151_koizumi-jyunichiro     0  0.854527
             speech topic     ratio
31  44_ikeda-hayato     1  0.996909
0    47_sato-eisaku     1  0.994826
75  41_ikeda-hayato     1  0.993283
4    49_sato-eisaku     1  0.993257
47   66_sato-eisaku     1  0.992834
                speech topic     ratio
51      187_abe-shinzo     2  0.906505
40  181_noda-yoshihiko     2  0.852242
13      183_abe-shinzo     2  0.820633
42  179_noda-yoshihiko     2  0.722379
1       185_abe-shinzo     2  0.717967
                 speech topic     ratio
41     144_obuchi-keizo     3  0.797251
74    121_kaifu-toshiki     3  0.584462
25     143_obuchi-keizo     3  0.563354
79    119_kaifu-toshiki     3  0.532736
62  125_miyazawa-kiichi     3  0.513640
                   speech topic     ratio
14   113_t

この結果を見ると、総理大臣に就任した時期が近い演説では、共通のトピックが大きな割合を占めているようです。これは、それぞれの時代特有の政治的・経済的課題があり、首相が代わっても、その課題は共有されていたと解釈できるのかもしれません。そもそも日本の戦後政治ではほぼ一貫して自民党が政権を担っているので、政党の違いによる政策の違いというのは所信表明演説においてはほとんど顕在化したことがなく、演説の趣旨の違いはほぼ時代的な課題によっているのかもしれません。
